#Uploading Data to ChromaDB

## Objective
Uploading MITRE ATT&CK and CISA Advisories data into Chroma, a high-performance vector database, using Python. Aligns with the MITRE embed project and supports Retrieval-Augmented Generation (RAG) in cybersecurity.

##Step 1: Set Up the Enviornment

In [1]:
!pip install chromadb pandas transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.7 MB/s et

##Step 2: Load the MITRE Att&CK Data

In [2]:
import pandas as pd

In [15]:
# Load the MITRE ATT&CK datasets
mitre_data = pd.read_csv('/content/mitreembed_master_Chroma.csv', sep='\t', on_bad_lines='skip')
print("MITRE Data Sample:\n", mitre_data.head())

MITRE Data Sample:
                                              Subject  \
0  Analyze network data for uncommon data flows (...   
1  Analyze network data for uncommon data flows (...   
2  Analyze network data for uncommon data flows (...   
3  Analyze network data for uncommon data flows (...   
4  #  Windows #\n\nMonitor for unexpected process...   

                                        filepath    Date  \
0      https://attack.mitre.org/techniques/T1001  4/5/24   
1  https://attack.mitre.org/techniques/T1001/001  4/5/24   
2  https://attack.mitre.org/techniques/T1001/002  4/5/24   
3  https://attack.mitre.org/techniques/T1001/003  4/5/24   
4      https://attack.mitre.org/techniques/T1003  4/5/24   

                                                Body Source  
0  Adversaries may obfuscate command and control ...  MITRE  
1  Adversaries may add junk data to protocols use...  MITRE  
2  Adversaries may use steganographic techniques ...  MITRE  
3  Adversaries may impersonate leg

In [6]:
cisa_data = pd.read_csv('CISA_combo_features_new.csv')
print("CISA Data Sample:\n", cisa_data.head())

CISA Data Sample:
                                              Subject  \
0  CISA Releases Malware Analysis Reports on Barr...   
1  CISA Adds One Known Exploited Vulnerability to...   
2  Adobe Releases Security Updates for Multiple P...   
3      Fortinet Releases Security Update for FortiOS   
4    Microsoft Releases August 2023 Security Updates   

                              Date  \
0  Wed, 09 Aug 2023 19:19:13 +0000   
1  Wed, 09 Aug 2023 16:23:59 +0000   
2  Tue, 08 Aug 2023 21:16:14 +0000   
3  Tue, 08 Aug 2023 19:14:52 +0000   
4  Tue, 08 Aug 2023 19:13:07 +0000   

                                                Body  \
0      CISA Releases Malware Analysis Reports on ...   
1      CISA Adds One Known Exploited Vulnerabilit...   
2      Adobe Releases Security Updates for Multip...   
3      Fortinet Releases Security Update for Fort...   
4      Microsoft Releases August 2023 Security Up...   

                                            filepath  
0  /Users/Jupiter/Deskt

## Step 3: Initialize the Sentence Transformer

In [7]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.6 MB/s eta 0:00:00


In [8]:
#sentence embedding model
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [9]:
# This line downloads the 'all-MiniLM-L12-v2' Sentence Transformer model
original_model = SentenceTransformer('all-MiniLM-L12-v2')

# reload model using langchain wrapper
# This line saves the downloaded model to the current directory
original_model.save('./')

# Define the path where the embedding model is saved
embedding_model_path = './'
# Initialize HuggingFaceEmbeddings with the local model path
# This wraps the Sentence Transformer model for use with LangChain
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_path)

# Define example sentences to be embedded
sentences = ["This is an example sentence", "Each sentence is converted"]

# Re-initialize the Sentence Transformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
# Encode the example sentences into embeddings
embeddings = model.encode(sentences)

# Re-initialize HuggingFaceEmbeddings again
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_path)

# Print the generated embeddings
print(embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-9-3142360160.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_path)


[[-2.02660376e-04  8.14801678e-02  3.13617773e-02  2.92065088e-03
   2.61564665e-02  2.90739723e-02  7.82618299e-02 -1.80417008e-03
   1.01344392e-01 -4.51711752e-02  5.84349968e-02 -1.53200626e-02
   5.49956299e-02 -9.86434743e-02 -3.50253358e-02  8.45681224e-03
   1.58608723e-02  1.05626173e-02 -3.42710204e-02 -4.75065550e-03
   9.99021381e-02 -2.06017476e-02 -4.47837450e-02  3.12135555e-02
  -1.19240899e-02 -5.15015051e-02 -1.33605618e-02  1.89622138e-02
   9.76809859e-02 -5.44110276e-02 -3.43314074e-02  8.12904835e-02
   4.88119870e-02 -1.10284016e-02  2.13519260e-02  1.27190053e-02
  -1.43967532e-02  3.62864546e-02 -7.61233121e-02  3.23293582e-02
   2.08102744e-02 -4.22016494e-02  9.12907198e-02  2.08530556e-02
  -3.08016893e-02 -8.38504806e-02  1.30890906e-02 -3.00631300e-02
   4.11228687e-02 -1.27495304e-01 -7.78027922e-02 -3.93412225e-02
   1.52596645e-03 -2.80108545e-02  3.41662504e-02  1.46713192e-02
  -7.71654397e-02  1.63619593e-01  4.11295556e-02 -5.24459854e-02
  -4.18772

 ## Step 4: Setup ChromaDB

In [10]:
# Import necessary libraries
from langchain.vectorstores import Chroma  # For interacting with the Chroma vector database
import chromadb  # The ChromaDB client library
from sentence_transformers import SentenceTransformer # For creating sentence embeddings

from langchain.document_loaders import DataFrameLoader # To load data from pandas DataFrames into a document format
from langchain.embeddings import HuggingFaceEmbeddings # To use HuggingFace models for creating embeddings
import re # Regular expression operations

import pandas as pd # For data manipulation and analysis, especially with DataFrames

In [18]:
# define logic for embeddings storage
# Define the path for storing ChromaDB data (in the current directory)
chromadb_path = './'
# Initialize the ChromaDB client
chroma_client = chromadb.Client()
# Print the version of the ChromaDB client
print(chroma_client.get_version())

# Handle missing values in the 'Body' column
mitre_data['Body'] = mitre_data['Body'].fillna('')

# assemble product documents in required format (id, text)
# Create a DataFrameLoader instance, specifying that the 'Body' column contains the document content
loader = DataFrameLoader(
    mitre_data,
    page_content_column='Body'
    )

1.0.15


In [19]:
# Load the data from the mitre_data DataFrame into a list of Document objects
documents = loader.load()

## Step 5: Define Logic for embeddings storage

In [21]:
chromadb_path = './'

# Create a Chroma vector store from the documents and embeddings
vectordb = Chroma.from_documents(
  documents=documents, # The list of documents to embed and store
  embedding=embedding_model, # The embedding model to use
  persist_directory=chromadb_path, # The directory where the ChromaDB data will be stored
  collection_name = 'CISA_MITRE' # Optional: specify a name for the collection
  )

# persist vector db to storage
vectordb.persist()

/tmp/ipython-input-21-2309483926.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


## Step 6: Check the number of collections in the vectorDB

In [22]:
#count documents
vectordb._collection.count()

3628

## Step 7: Query ChromaDB for MITRE-related

In [23]:
query_text = "remote desktop attack"
vectordb.similarity_search_with_score(query_text)
#Response: Rows that closely aligns with remote desktop attack, meta data of row
# and file path which includes the technique

[(Document(metadata={'Date': '4/5/24', 'Subject': 'Consider monitoring processes for `tscon.exe` usage and monitor service creation that uses `cmd.exe /k` or `cmd.exe /c` in its arguments to detect RDP session hijacking.\n\nUse of RDP may be legitimate, depending on the network environment and how it is used. Other factors, such as access patterns and activity that occurs after a remote login, may indicate suspicious or malicious behavior with RDP.', 'filepath': 'https://attack.mitre.org/techniques/T1563/002', 'Source': 'MITRE'}, page_content="Adversaries may hijack a legitimate user‚Äôs remote desktop session to move laterally within an environment. Remote desktop is a common feature in operating systems. It allows a user to log into an interactive session with a system desktop graphical user interface on a remote system. Microsoft refers to its implementation of the Remote Desktop Protocol (RDP) as Remote Desktop Services (RDS).(Citation: TechNet Remote Desktop Services)\n\nAdversari

## Step 8: Inspect a record

In [24]:
#examine a vector db record
#shows a random record where metadata matches the query text
rec = vectordb._collection.peek(1)
# Print the metadata of the record
print('Metadata:  ', rec['metadatas'])
# Print the document content of the record
print('Documents:  ', rec['documents'])
# Print the IDs of the records
print('ids:        ', rec['ids'])
# Print the embeddings of the records
print('embeddings: ', rec['embeddings'])

Metadata:   [{'filepath': 'https://attack.mitre.org/techniques/T1001', 'Source': 'MITRE', 'Date': '4/5/24', 'Subject': 'Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server). Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. Analyze packet contents to detect communications that do not follow the expected protocol behavior for the port that is being used. (Citation: University of Birmingham C2)'}]
Documents:   ['Adversaries may obfuscate command and control traffic to make it more difficult to detect. Command and control (C2) communications are hidden (but not necessarily encrypted) in an attempt to make the content more difficult to discover or decipher and to make the communication less conspicuous and hide commands from being seen. This encompasses many methods, such as adding junk data to protocol traffic, using steganography, or imperso

## Step 9: Real-World Use Cases
By integrating ChromaDB with MITRE ATT&CK data, cybersecurity analysts can:

1. Rapidly map alerts to known techniques.
2. Cross-reference threat intelligence feeds.

## Conclusion
By leveraging ChromaDB and RAG frameworks, we can transform static cybersecurity data into dynamic, actionable insights.